# Лабораторна робота 3. Групи, порядок елемента в групі

_Примітка_: наведений код є лише одним з можливих шаблонів виконання. Можете писати по-своєму, але розділяйте свій код на функції, щоб їх можна було простіше перевіряти.

## Завдання 1. 

__Задайте групи $D_{10}$, $Z_{16}$, $A_5$, $SL(2, \mathbb{Z}_3)$*. Для кожної з них виведіть таблицю Келі, знайдіть порядок групи, перевірте чи група абелева. Перевірте чи будуть ізоморфними групи $SL(2, Z_3)$ та $D_{12}$.__

*: група матриць 2х2 з визначником 1 над полем $\mathbb{Z}_3$

In [ ]:
from sage.matrix.operation_table import OperationTable
G1 = DihedralGroup(10)
G2 = CyclicPermutationGroup(16)
G3 = AlternatingGroup(5)
G4 = SL(2, Integers(3))
groups = {G1 : "D_10",G2 : "Z_16",G3 : "A_5",G4 : "SL(2, Z3)",}
    
with open("3_1.txt", "w") as f:
    for group in groups.keys():
        print(f"Group: {groups[group]}", file = f)
        print(OperationTable(group, operator.mul), file = f)
        print(f"Group order: {group.cardinality()}", file = f)
        print(f"The group is abelian: {group.is_abelian()}", file = f)
        print("\n", file = f)
    print(f"D12 is isomorphic to SL(2, Z3): {G4.is_isomorphic(DihedralGroup(12))}", file = f)
  

## Завдання 2. 

__Знайдіть к-ть елементів кожного можливого порядку в групах $S_{100}$ та $A_{100}$.__

-----
___Зауваження___: вивід буде великий, збережіть його в окремий текстовий файл

___Зауваження 2___: оцініть спершу к-ть елементів в групі, а потім ще раз подумайте чи варто тут писати повний перебір

In [56]:
from sage.arith.functions import LCM_lis
def length_amount(partition):
    """Нам треба знайти кількість циклів певного порядку
    """
    counter = 0
    current_length = partition[0]
    for length in partition:
        if length != current_length:
            current_length = length
            yield counter
            counter = 1
        else:
            counter += 1
    yield counter

factorials = {0: 1}
res = 1
for i in range(1, 101):
    res *= i
    factorials[i] = res
    
def number(partition, n):
    global factorials
    res = 1
    for length in partition:
        res *= factorials[n] / factorials[n - length] / length
        n -= length
    for amount in length_amount(partition):
        if amount == 1:
            pass
        else:
            res /= factorials[amount]
    return res

def order_amount(n):
    order_amount = {}
    unique_partitions = Partitions(n)
    for partition in unique_partitions:
        ord_ = LCM_list(partition)
        if ord_ not in order_amount:
            order_amount[ord_] = number(partition, n)
        else:
            order_amount[ord_] += number(partition, n)
    keys = list(order_amount.keys())
    keys.sort()
    return {i:order_amount[i] for i in keys}

with open(f"orders_s100.txt", "w") as f:
    counter = 0
    for order, amount in order_amount(n).items():
        print(f"Order: {order}, number of elements: {amount}", file = f)
        counter += amount



In [ ]:
from sage.arith.functions import LCM_list
def is_even(partition):
    """Так як у нас в групі А100 є половина елементів групи S100, то нам треба знайти всі
    елементи парного порядку
    """
    counter = 0
    for length in partition:
        if length % 2 == 1:
            pass
        else:
            counter += 1
    if counter % 2 == 0:
        return True
    else:
        return False

def order_amount(n):
    order_amount = {}
    unique_partitions = Partitions(n)
    for partition in unique_partitions:
        if not is_even(partition):
            continue
        else:
            ord_ = LCM_list(partition)
            if ord_ not in order_amount:
                order_amount[ord_] = number(partition, n)
            else:
                order_amount[ord_] += number(partition, n)
    keys = list(order_amount.keys())
    keys.sort()
    return {i:order_amount[i] for i in keys}

with open(f"orders_a100.txt", "w") as f:
    counter = 0
    for order, amount in order_amount(n).items():
        print(f"Order: {order}, number of elements: {amount}", file = f)
        counter += amount



## Завдання 3. 
___Для заданих натуральних $n, k$ ($1 <= n \le 1 000 000, 1 \le k \le n!$) визначте чи існує в групі $S_n$ елемент порядку $k$.___

----------

___Зауваження:___ зверніть увагу на межі, в яких задано $n, k$. 

In [ ]:
def exists(k, n):
    if factorial(n) % k != 0:
        return False, None
    divs = iter(divisors(k))
    possible_lengths = []
    for length in divs:
        if sum(possible_lengths) > n:
            break
        elif length > n:
            break
        else:
            possible_lengths += [length]
    candidates = iter(Subsets(possible_lengths))
    for candidate in candidates:
        if sum(candidate) > n:
            continue
        if LCM_list(candidate) != k:
            continue
        else:
            return True, candidate
    return False, None